In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])
df_ho=df_ho.reset_index()
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [3]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [4]:
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv', 'train')
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv', 'test')

train Done.
test Done.


In [5]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [6]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [7]:
t_train = train.copy()
t_test = test.copy()

In [8]:
y = t_train['target'] 

x = t_train.drop(['target','id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

t_test = t_test.drop(['id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)
t_test = t_test.drop(['base_date','start_turn_restricted','end_turn_restricted','start_latitude','start_longitude','end_latitude','end_longitude'], axis=1)
print(x.shape)
print(y.shape)
print(t_test.shape)

(4701217, 11)
(4701217,)
(291241, 4)


##### for i in df_ho['locdate']:
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in df_ho['locdate']:
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    

In [10]:
x = x.replace('월', 1)
x = x.replace('화', 2)
x = x.replace('수', 3)
x = x.replace('목', 4)
x = x.replace('금', 5)
x = x.replace('토', 6)
x = x.replace('일', 7)
t_test = t_test.replace('월', 1)
t_test = t_test.replace('화', 2)
t_test = t_test.replace('수', 3)
t_test = t_test.replace('목', 4)
t_test = t_test.replace('금', 5)
t_test = t_test.replace('토', 6)
t_test = t_test.replace('일', 7)

In [11]:
x = x.drop(['base_date','start_turn_restricted','end_turn_restricted','start_latitude','start_longitude','end_latitude','end_longitude'], axis=1)

In [12]:
x = x.to_numpy()
y = y.to_numpy()

In [13]:
t_test =t_test.to_numpy()

In [14]:
x

array([[ 4., 17.,  1., 60.],
       [ 4., 21.,  2., 60.],
       [ 7.,  7.,  2., 80.],
       ...,
       [ 1., 22.,  2., 60.],
       [ 3.,  2.,  2., 80.],
       [ 2.,  6.,  2., 60.]])

In [15]:
y

array([52., 30., 61., ..., 30., 73., 35.])

In [16]:
train_input, val_input, train_target, val_target = train_test_split(
    x, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [18]:
xg_reg = xgb.XGBRegressor(random_state=42).fit(train_input, train_target)

In [19]:
pred = xg_reg.predict(t_test)

In [20]:
sample_submission = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/sample_submission.csv')

In [21]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/submit.csv", index = False)

In [22]:
sample_submission

,id,target
0,TEST_000000,27.979931
1,TEST_000001,48.433704
2,TEST_000002,53.601021
3,TEST_000003,41.352516
4,TEST_000004,28.378208
...,...,...
291236,TEST_291236,49.231049
291237,TEST_291237,33.793175
291238,TEST_291238,29.710442
291239,TEST_291239,36.406677


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

C:\Users\82108\anaconda3\envs\base2\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)